# NMF Preprocessing for player recomendations

## Overview - 

## 4.1 - NMF model explanation

## 4.2 - NMF hyperparameter tuning

### 4.2.1 Tuning max_iter 

### 4.2.2 Tuning number of skill set groups "c"

## 4.3 - Model output 

### 4.3.1 Label Weight matrix skill set groups

### 4.3.2 Create ranking data from from skill set values

### 4.3.3 Save output and model 


In [1]:
## import packages and tools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline
from utils import save_file

%matplotlib inline

In [2]:
## load data
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [3]:
## set index and drop negative and categotical columns 
X = df.set_index('player_name')

# remove categorical and features with negative values
X = X.drop(columns= ['Base Salary','Player', 'Club',
                     'Nation', 'Position','fb_id', 'Squad_from_op','G-xG','np:G-xG'])
X.rename(columns={'prog_carry+lng_comp+crosses':'Progressive_play'},inplace=True)
X.shape

(705, 100)

In [5]:
## select key features to input into NMF 
X = X[[ 'Nineties', 'xA',
       'Key_pass', 'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
        'Medium_Cmp', 'Medium_Att', 
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct', 'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
       'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
       'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
       'PK_won', 'Aerial_win_pct', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
       'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG',
       'npxG/Sh', 'pAdj_Total_tackles', 'pAdj_Tackles_Won',
       'pAdj_Tackles_Def_3rd', 'pAdj_Tackles_Mid_3rd', 'pAdj_Tackles_Att_3rd',
       'pAdj_Num_Dribblers_tackled', 'pAdj_Num_Dribbled_past', 
       'pAdj_Blocked_shots', 'pAdj_Blocked_SOT', 'Tackle_pct_won','pAdj_Blocked_pass',
       'pAdj_Interceptions', 'pAdj_Tackles_and_Ints', 'pAdj_Clearences',
      'pAdj_Red_cards','pAdj_Fouls', 'pAdj_def_interceptions', 'pAdj_Recoveries',
       'pAdj_Aerial_Duels_lost', 'lost_tackles', 'True_tackle_pct',
       'pAdj_Tackle_int_blocks', 'avg_shrt+med_pass_pct', 'pct_long_balls',
       'Progressive_play', 'attacking_touches']]
# work with players who have played at least 5 games 
X = X.loc[(X['Nineties']>5)]
X.shape

(541, 85)

In [6]:
# save X matrix 
X.to_csv('../data/NMF_X.csv')

# 4.1 - NMF model explanation - Non Negative Matrix Factorization 

NMF or Non Negative Matrix Factorization is widely used for topic modeling and document clustering but there are many examples of NMF being used to cluster other kinds of datasets such as classifying Companies on extra financial criteria.https://towardsdatascience.com/using-nmf-to-classify-companies-a77e176f276f . I am going to use NMF to classify my MLS 2021 data set by the "topic" or ,grouping of player stats,that I will call "skill set group". I will use the skill sets to classify players in the league and then use a normalized version of the decomposed matrix to identify players who are closest to Key players Identified in my EDA . At the end I should have all players classified by a key skill set group. Players closest to the target players identified show how franchises could use NMF to Identify other players that fit into roles played by players in the MLS. 

# X matrix - Original player matrix
With the player matrix X being of n players(rows) and f features(player stats) (n x f)


## Weight matrix - weights of each row for each skill set group or topic 
c = number of Skill Set Groups that will be generated by NMF 
This matrix has rows filled by players  with the skill set features generated  by the NMF algorithm as the columns give you the weight matrix. 
W = n x c 

## Hidden layer matrix - hidden layer of feature values for each skill set group
This matrix has rows of players features with  skill set groups as the columns. 
H = f x c

## The matrix multiplication of W (weight)* H(hidden) approximates the original X player matrix

The two matixies W and H when multiplied togeather approximates the original X matrix. This allows player data to be observed in a lower dimensional space by comparing player skill set values in the weight matrix.  

## The W weight matrix is used to find players closest to the target player in question

The distance between the topic values of each player in the W matrix can be used to measure who is the most simular allowing for recommendations of players closest to the target player to be created.

The skill set group that has the highest ranking for each player is used to classify or label groups of players who are Rows filled by players  with the skill set features generated  by the NMF algorithm as the columns gives you the weight matrix,  W = n x c.similar in their player stats features. 



In [6]:
# function to viz skill groups and players most associated with skill group

def display_features(H,W,feature_names, X_matrix ,no_top_features, no_top_players):
    """ visualize skill get group and highest ranked players in group """
    topics = {}
    # iterate through topics in topic-term matrix, 'H' aka
    # H is the hidden layer which is shape (F x C) feature times topic matrix
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_features - 1:-1]]))
        
        # add features to topics dictionary for later assesment. 
        
        topics[topic_idx] = [ (feature_names[i] + " (" + str(topic[i].round(2)) + ")") 
                             for i in topic.argsort()[:-(no_top_features+3) - 1:-1]]
        
        top_player_indicies = np.argsort( W[:,topic_idx] )[::-1][0:no_top_players]
        for p_index in top_player_indicies:
            
            print(p_index," ",X_matrix.index[p_index])
    
    return topics 


def add_skill_group(X,W):
    """ add skill group to player feature to use for classification"""
    df_new = X.copy()
    # Get the top predicted topic and add to df copy 
    df_new['pred_topic_num']= [np.argsort(each)[::-1][0] for each in W]
  
    return df_new

## 4.2 - NMF hyperparamter tuning

### 4.2.1 tuning of max_iter parameter to allow for model convergance 

In [7]:
# try increasing number of max_iter to allow for model convergance
for test in np.arange(100,1000,100):
    c = 11
    # Create a MaxAbsScaler: scaler
    transformer = MaxAbsScaler().fit(X)

    # scale data
    scaled_X= transformer.transform(X)

    # Create an NMF model: nmf

    nmf = NMF(n_components=c,max_iter=test,init='nndsvda',  random_state=42)

    W = nmf.fit_transform(scaled_X)
    H = nmf.components_
    err = nmf.reconstruction_err_

    print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)           


# topics = display_features(H,W,feature_names, X, no_top_features,no_top_players)

/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


NMF(init='nndsvda', max_iter=100, n_components=11, random_state=42) error : 15.286725358664198 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', n_components=11, random_state=42) error : 15.215542645986101 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=300, n_components=11, random_state=42) error : 15.198796106203938 
  W shape (541, 11) H shape  (11, 85)


/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 400 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


NMF(init='nndsvda', max_iter=400, n_components=11, random_state=42) error : 15.189526288439696 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=500, n_components=11, random_state=42) error : 15.184543166408027 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=600, n_components=11, random_state=42) error : 15.182825590282299 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=700, n_components=11, random_state=42) error : 15.182825590282299 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=800, n_components=11, random_state=42) error : 15.182825590282299 
  W shape (541, 11) H shape  (11, 85)
NMF(init='nndsvda', max_iter=900, n_components=11, random_state=42) error : 15.182825590282299 
  W shape (541, 11) H shape  (11, 85)


### max_iter tuning 
- more than 300 iterations required to converge the model. Will increase to 500 iterations. 

## 4.2.2 tuning number of skill set groups "c"

### 4.2.2.1 Inital NMF attempt with c = 11 skill group topics 

In [8]:

c = 11
# Create a MaxAbsScaler: scaler
transformer = MaxAbsScaler().fit(X)

# scale data
scaled_X= transformer.transform(X)

# Create an NMF model: nmf

nmf = NMF(n_components=c,max_iter=600,init='nndsvda', random_state=42)

W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)

            
feature_names = X.columns.values
no_top_features = 4
no_top_players = 3

topics = display_features(H,W,feature_names, X, no_top_features,no_top_players)

NMF(init='nndsvda', max_iter=600, n_components=11, random_state=42) error : 15.182825590282299 
  W shape (541, 11) H shape  (11, 85)
Topic 0:
recept_pct (12.38) avg_shrt+med_pass_pct (11.95) pct_long_balls (10.48) touches_def_box (10.48)
488   Brad Guzan
518   Maxime Crépeau
526   Zac MacMath
Topic 1:
dribble_Succ (3.92) num_dribble_past (3.79) dribble_Att (3.57) dispossed (3.21)
51   Yeferson Soteldo
136   Brian Rodríguez
2   Emanuel Reynoso
Topic 2:
pAdj_Tackles_Mid_3rd (3.86) pAdj_Tackles_Won (3.44) pAdj_Total_tackles (3.41) pAdj_Tackles_and_Ints (2.88)
409   Judson
462   Franco Ibarra
246   Eric Remedi
Topic 3:
xA (1.9) Key_pass (1.71) touches_att_3rd (1.56) Comp_prog (1.48)
2   Emanuel Reynoso
0   Carles Gil
6   Lucas Zelarayán
Topic 4:
pAdj_Clearences (2.14) pAdj_Blocked_shots (1.81) touches_def_3rd (1.46) Aerial_win_pct (1.28)
426   Nathan Cardoso
392   Jon Bell
348   Alan Franco
Topic 5:
Total_Cmp_pct (1.3) avg_shrt+med_pass_pct (1.26) dribble_Succ_pct (1.12) recept_pct (1.07)

In [9]:
# check groups 
df_new = add_skill_group(X,W)
df_new['pred_topic_num'].value_counts()

7     153
10    115
8      97
9      78
5      57
4      24
6       8
0       6
3       3
Name: pred_topic_num, dtype: int64

### 2.2.1 Tuning Insight 
- 11 skill groups lead to only 9 clusters shown above. I will reduce c to be able to have more balanced clusters. 
- The highest rated players for each skill set are related in playing style showing that the associations between players built out by NMF look promising 

## 2.2.2 - Attemping c('skill groups') = 9 to look for balanced groups.  

In [10]:
c = 9

nmf = NMF(n_components=c,max_iter=1000,init='nndsvda', l1_ratio=.05, random_state=42)

W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)
df_new = add_skill_group(X,W)
print(df_new['pred_topic_num'].value_counts())
            
feature_names = X.columns.values
no_top_features = 4
no_top_players = 5

topics = display_features(H,W,feature_names, X, no_top_features,no_top_players)

NMF(init='nndsvda', l1_ratio=0.05, max_iter=1000, n_components=9,
    random_state=42) error : 16.09706567324863 
  W shape (541, 9) H shape  (9, 85)
8    103
7     95
5     87
6     76
4     68
3     49
2     43
0     18
1      2
Name: pred_topic_num, dtype: int64
Topic 0:
recept_pct (9.29) avg_shrt+med_pass_pct (8.92) pct_long_balls (8.13) touches_def_box (7.91)
488   Brad Guzan
518   Maxime Crépeau
526   Zac MacMath
530   David Ochoa
428   J.T. Marcinkowski
Topic 1:
dribble_Succ (2.78) num_dribble_past (2.71) dribble_Att (2.59) dispossed (2.38)
51   Yeferson Soteldo
2   Emanuel Reynoso
136   Brian Rodríguez
6   Lucas Zelarayán
0   Carles Gil
Topic 2:
pAdj_Clearences (2.35) pAdj_Blocked_shots (2.03) touches_def_3rd (1.79) pAdj_def_interceptions (1.47)
426   Nathan Cardoso
348   Alan Franco
417   Daniel Steres
365   Rudy Camacho
422   Francisco Calvo
Topic 3:
Total_Cmp_pct (1.69) avg_shrt+med_pass_pct (1.64) recept_pct (1.42) Long_Cmp_pct (1.26)
499   Kamohelo Mokotjo
459   Ralph Pris

## 4.3 - Model output 

### 4.3.1 Label Weight matrix skill set groups
- use top features in each skill set group to create skill set names.

In [11]:
skill_sets = pd.DataFrame(topics )
columns =['Defensive passing','Dribbling','Defensive actions','Short passing','Playing time','Tackling','Attacking play','Passing volume','Progressive passing']
skill_sets.columns = columns
skill_sets

,Defensive passing,Dribbling,Defensive actions,Short passing,Playing time,Tackling,Attacking play,Passing volume,Progressive passing
0,recept_pct (9.29),dribble_Succ (2.78),pAdj_Clearences (2.35),Total_Cmp_pct (1.69),Nineties (2.11),pAdj_Tackles_Mid_3rd (1.1),touches_att_box (1.38),touches_mid_3rd (0.89),Crosses (1.06)
1,avg_shrt+med_pass_pct (8.92),num_dribble_past (2.71),pAdj_Blocked_shots (2.03),avg_shrt+med_pass_pct (1.64),Total_Cmp_pct (1.69),pAdj_Total_tackles (0.98),SoT (1.31),recep_succ (0.85),touches_att_3rd (0.69)
2,pct_long_balls (8.13),dribble_Att (2.59),touches_def_3rd (1.79),recept_pct (1.42),avg_shrt+med_pass_pct (1.63),pAdj_Tackles_Won (0.97),npxG (1.24),Total_Cmp (0.77),attacking_touches (0.64)
3,touches_def_box (7.91),dispossed (2.38),pAdj_def_interceptions (1.47),Long_Cmp_pct (1.26),dribble_Succ_pct (1.45),pAdj_Tackles_and_Ints (0.84),xG (1.19),Total_Att (0.77),Short_Att (0.53)
4,Long_Att (7.82),carries_prog (2.01),pAdj_Interceptions (1.47),dribble_Succ_pct (1.25),Long_Cmp_pct (1.39),pAdj_Num_Dribbled_past (0.82),Sh (1.19),carries (0.77),xA (0.51)
5,Total_Prg_Dist (7.7),attacking_touches (1.89),Aerial_win_pct (1.38),Dist (1.14),recept_pct (1.39),pAdj_Num_Dribblers_tackled (0.81),Offsides (1.04),recep_targ (0.74),recept_pct (0.5)
6,touches_def_3rd (7.13),dribble_megs (1.88),pAdj_Tackle_int_blocks (1.37),Short_Att (0.86),Short_Att (0.7),pAdj_Tackle_int_blocks (0.77),Gls (1.03),Total_Tot_Dist (0.73),pAdj_Blocked_pass (0.5)


### 4.3.2 Create ranking data from from skill set values

In [12]:
dfw= pd.DataFrame(W,columns=columns,index = X.index)
dfw.head()

,Defensive passing,Dribbling,Defensive actions,Short passing,Playing time,Tackling,Attacking play,Passing volume,Progressive passing
player_name,,,,,,,,,
Carles Gil,0.013192,0.217938,0.000000,0.0,0.172427,0.000000,0.084553,0.571970,0.501718
Julian Gressel,0.025871,0.031960,0.042472,0.0,0.118228,0.103026,0.111820,0.205170,0.728908
Emanuel Reynoso,0.002543,0.292327,0.000000,0.0,0.212190,0.279362,0.122834,0.539871,0.203679
Albert Rusnák,0.014539,0.076447,0.000000,0.0,0.323945,0.000000,0.176914,0.354539,0.279889
Maximiliano Moralez,0.009812,0.057264,0.000000,0.0,0.229219,0.189130,0.243082,0.528513,0.402812


### Use skill Set group values to create ranks to use for clustering and comparisons

In [90]:
for col in dfw.columns.values:
    title = col +' rank'
    dfw[title] = dfw[col].rank(pct=True,na_option = 'bottom',ascending=True)
dfw[['Defensive actions','Defensive actions rank']].nlargest(5,columns='Defensive actions rank')

,Defensive actions,Defensive actions rank
player_name,,
Nathan Cardoso,0.355321,1.000000
Alan Franco,0.315812,0.998152
Daniel Steres,0.308983,0.996303
Rudy Camacho,0.305769,0.994455
Francisco Calvo,0.301305,0.992606


### Defensive action rank insight 
- SJE new defenive signing Francisco Calvo comes up in top 5 rank with Nathan Cordoso for the defensive actions skill set group

## 4.3.3 Saving dataframes and models 
- save rank matrix
- save normalized Weight matrix for cosine simularity
- save pickled nmf model 

In [93]:
# save dfw for modeling and coparison

dfw.to_csv('../data/W_ranks.csv', encoding='utf-8', index=False)
skill_sets.to_csv('../data/skill_sets.csv', encoding='utf-8', index=False)

In [92]:
# save normalized W matrix for calculating cosine simularity between players

transformer = Normalizer().fit(W)
normed_W= transformer.transform(W)
# can add back club and salary here 
normed_W = pd.DataFrame(normed_W,index=X.index.values, columns=columns)

normed_W.to_csv('../data/normed_W.csv',encoding='utf-8')

In [15]:
## save nmf model for later use
            
save_file(data= nmf, fname='nmf.pkl',dname='../models/')

A file already exists with this name.



Do you want to overwrite? (Y/N) Y


Writing file.  "../models/nmf.pkl"
